In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.linear_model import LogisticRegression
from sklearn.cluster import KMeans
import statsmodels.api as sm

# Step 1: Setup
trial_pp_dir = "trial_pp"
trial_itt_dir = "trial_itt"
os.makedirs(trial_pp_dir, exist_ok=True)
os.makedirs(trial_itt_dir, exist_ok=True)


In [2]:
# Step 2: Data Preparation
df = pd.read_csv("data/data_censored.csv")
print(df.head())
print(df.describe())

# Fixing ID issue: Ensure IDs are not just 1 and 2
df["id"] = df.groupby("id").ngroup() + 1

# Per-protocol
trial_pp = {
    "data": df,
    "id": "id",
    "period": "period",
    "treatment": "treatment",
    "outcome": "outcome",
    "eligible": "eligible"
}

# ITT
trial_itt = {
    "data": df,
    "id": "id",
    "period": "period",
    "treatment": "treatment",
    "outcome": "outcome",
    "eligible": "eligible"
}

print("Trial Sequence Object")
print("Estimand: Per-Protocol")
print(f"Data: \n##  - N: {df.shape[0]} observations from {df['id'].nunique()} patients")
print(df.head(10).to_string(index=False))

print("Estimand: Intention-to-Treat")
print(f"Data: \n##  - N: {df.shape[0]} observations from {df['id'].nunique()} patients")
print(df.head(10).to_string(index=False))



   id  period  treatment  x1        x2  x3        x4  age     age_s  outcome  \
0   1       0          1   1  1.146148   0  0.734203   36  0.083333        0   
1   1       1          1   1  0.002200   0  0.734203   37  0.166667        0   
2   1       2          1   0 -0.481762   0  0.734203   38  0.250000        0   
3   1       3          1   0  0.007872   0  0.734203   39  0.333333        0   
4   1       4          1   1  0.216054   0  0.734203   40  0.416667        0   

   censored  eligible  
0         0         1  
1         0         0  
2         0         0  
3         0         0  
4         0         0  
               id      period   treatment          x1          x2          x3  \
count  725.000000  725.000000  725.000000  725.000000  725.000000  725.000000   
mean    49.278621    7.051034    0.467586    0.405517   -0.173552    0.486897   
std     28.119313    5.802351    0.499293    0.491331    0.997552    0.500173   
min      1.000000    0.000000    0.000000    0.0000

In [5]:
# Step 3: Weight models and censoring
# Step 3.1: Set Switch Weight Model
def fit_logit_model(formula, df, save_path):
    """Fits a logistic regression model, saves it, and returns the predicted probabilities."""
    model = sm.Logit.from_formula(formula, data=df).fit()
    model.save(save_path)
    return model.predict(df)

# Numerator Model (age)
df["switch_numerator"] = fit_logit_model("treatment ~ age", df, os.path.join(trial_pp_dir, "switch_numerator.pkl"))

# Denominator Model (age + x1 + x3)
df["switch_denominator"] = fit_logit_model("treatment ~ age + x1 + x3", df, os.path.join(trial_pp_dir, "switch_denominator.pkl"))

# Compute Stabilized Weights
df["switch_weight"] = df["switch_numerator"] / df["switch_denominator"]

# Save processed dataset
df.to_csv(os.path.join(trial_pp_dir, "processed_data.csv"), index=False)

# Print summary
print("Switch weights computed successfully.")
print(df[["switch_numerator", "switch_denominator", "switch_weight"]].head())

# Step 3.2: Set Censor Weight Model

def fit_logit_model(formula, df, save_path):
    """Fits a logistic regression model, saves it, and returns the predicted probabilities."""
    model = sm.Logit.from_formula(formula, data=df).fit()
    model.save(save_path)
    return model.predict(df)

# Per-Protocol (PP) Estimand - No Pooling
df["censor_numerator_pp"] = fit_logit_model("censored ~ x2", df, os.path.join(trial_pp_dir, "censor_numerator.pkl"))
df["censor_denominator_pp"] = fit_logit_model("censored ~ x2 + x1", df, os.path.join(trial_pp_dir, "censor_denominator.pkl"))

# Compute IPCW for Per-Protocol
df["censor_weight_pp"] = df["censor_numerator_pp"] / df["censor_denominator_pp"]

# Intention-to-Treat (ITT) Estimand - Pooling by Numerator
df["censor_numerator_itt"] = fit_logit_model("censored ~ x2", df, os.path.join(trial_itt_dir, "censor_numerator.pkl"))
df["censor_denominator_itt"] = fit_logit_model("censored ~ x2 + x1", df, os.path.join(trial_itt_dir, "censor_denominator.pkl"))

# Compute IPCW for ITT
df["censor_weight_itt"] = df["censor_numerator_itt"] / df["censor_denominator_itt"]

# Save processed dataset
df.to_csv(os.path.join(trial_pp_dir, "processed_data.csv"), index=False)
df.to_csv(os.path.join(trial_itt_dir, "processed_data.csv"), index=False)

# Print summary
print("Censor weights computed successfully.")
print(df[["censor_weight_pp", "censor_weight_itt"]].head())

Optimization terminated successfully.
         Current function value: 0.662406
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.660234
         Iterations 5
Switch weights computed successfully.
   switch_numerator  switch_denominator  switch_weight
0          0.592014            0.636513       0.930088
1          0.581815            0.626528       0.928634
2          0.571545            0.549849       1.039459
3          0.561214            0.539206       1.040816
4          0.550830            0.595948       0.924292
Optimization terminated successfully.
         Current function value: 0.271311
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.267425
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.271311
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.267425
         Iterations 7
Censor weights compu

In [7]:
import pickle

# Function to load and display logistic regression model summaries
def show_weight_model(model_path):
    """Loads a saved logistic regression model and prints its summary."""
    with open(model_path, "rb") as file:
        model = pickle.load(file)
    print(model.summary())

# Display Per-Protocol Weight Models
print("\nWeight Models for Per-Protocol (PP) Censoring:")
show_weight_model(os.path.join(trial_pp_dir, "censor_numerator.pkl"))
show_weight_model(os.path.join(trial_pp_dir, "censor_denominator.pkl"))

print("\nWeight Models for Intention-to-Treat (ITT) Censoring:")
show_weight_model(os.path.join(trial_itt_dir, "censor_numerator.pkl"))
show_weight_model(os.path.join(trial_itt_dir, "censor_denominator.pkl"))

print("\nWeight Models for Treatment Switching:")
show_weight_model(os.path.join(trial_pp_dir, "switch_numerator.pkl"))
show_weight_model(os.path.join(trial_pp_dir, "switch_denominator.pkl"))



Weight Models for Per-Protocol (PP) Censoring:
                           Logit Regression Results                           
Dep. Variable:               censored   No. Observations:                  725
Model:                          Logit   Df Residuals:                      723
Method:                           MLE   Df Model:                            1
Date:                Sun, 09 Mar 2025   Pseudo R-squ.:                 0.02676
Time:                        20:08:06   Log-Likelihood:                -196.70
converged:                       True   LL-Null:                       -202.11
Covariance Type:            nonrobust   LLR p-value:                  0.001007
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -2.4481      0.141    -17.415      0.000      -2.724      -2.173
x2             0.4486      0.137      3.278      0.001       0.180       0.717
    